**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit_video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function takes as input the state of the game at t as well as the mode (training or evaluation) to return the action to be performed at t+1. This function use an ϵ-greedy strategy . This actually tackles the exploration-exploitation dilemma.

For the training two cases are possible.
Case 1: the action is chosen at random for exploration purposes. 
Case 2: the action is predicted from the state and level of knowledge of the network (poor at the beginning, rich at the end).
The parameter epsilon is essential in the choice of case 1 or 2. This is the ϵ-greedy strategy. We observe that if epsilon = 0, then we will always be in case 2 and if epsilon =1, then we will be always in case 1.

Epsilon is essential especially because if the return of the learned_act is poor and stuck in one action, the diversity will come from the case 1. That's why we call it discorvery parameter

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

position is an array of shape (grid_size x grid_size) representing where the rat can go. The rat is on case where the value is 1. It can not go on case where there is a -1. It can go on case where there is a 0.

board is an array of shape (grid_size x grid_size) representing the reward associated to each cell of the grid. The rat on case (x,y) will get the reward board (x,y). There is a -0.5 if there is a negative reward (malus), 1 if there is a positive reward (bonus) and 0 if there is no reward .

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
      return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        win = 0
        lose = 0
        game_over = False
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/12.0. Average score (0.0)
Win/lose count 11.5/9.0. Average score (1.25)
Win/lose count 8.0/11.0. Average score (-0.16666666666666666)
Win/lose count 7.5/11.0. Average score (-1.0)
Win/lose count 7.5/20.0. Average score (-3.3)
Win/lose count 8.5/16.0. Average score (-4.0)
Win/lose count 9.5/15.0. Average score (-4.214285714285714)
Win/lose count 10.0/11.0. Average score (-3.8125)
Win/lose count 8.0/13.0. Average score (-3.9444444444444446)
Win/lose count 14.5/22.0. Average score (-4.3)
Win/lose count 8.5/24.0. Average score (-5.318181818181818)
Win/lose count 11.0/14.0. Average score (-5.125)
Win/lose count 6.5/12.0. Average score (-5.153846153846154)
Win/lose count 9.0/10.0. Average score (-4.857142857142857)
Win/lose count 6.5/11.0. Average score (-4.833333333333333)
Win/lose count 4.5/9.0. Average score (-4.8125)
Win/lose count 10.5/20.0. Average score (-5.088235294117647)
Win/lose count 10.0/15.0. Average score (-5.083333333333333)
Win/lose count 10.5/8.0. Averag

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1/We have

$$ Q^{\pi}(s,a) = E_{\pi}[R_t |s_t = s, a_t = a] = E_\pi[\sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] = 
E_\pi[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a]
$$

and with a changement of variable $j +1 = k$ we have : 
$$
Q^{\pi}(s,a) = E_\pi[ r(s,a) + \sum_{j=0}^{\infty} \gamma^{j+1} r_{t+j+1}|s_t = s, a_t = a] \\ = r(s,a) + \gamma E_\pi[ \sum_{j=0}^{\infty} \gamma^{j} r_{t+j+1}|s_t = s, a_t = a]
$$

Thank to the propriety : $\mathbb E[X]=\mathbb E[\mathbb E[X|Y]]$, we have

$$
Q^{\pi}(s,a) = r(s,a) + \gamma  E_\pi[ E_\pi[ (\sum_{j=0}^{\infty} \gamma^{j} r_{t+j+1}|s_t = s, a_t = a)|s_{t+1} = s', a_{t+1} = a']] 
$$
$$
Q^{\pi}(s,a) = 1.r(s,a) + \gamma \sum_{(s',a')}p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{j=0}^{\infty}\gamma^j r_{(t+1)+j}|s_{t+1}=s',a_{t+1}=a']\\
$$

and because 
$$
\sum_{(s',a')}p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) = 1
$$
Then : 

$$
Q^{\pi}(s,a) = \sum_{(s',a')}p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) [r(s,a) + \gamma E[\sum_{j=0}^{\infty}\gamma^j r_{(t+1)+j}|s_{t+1}=s',a_{t+1}=a']]\\
= \sum_{(s',a')}p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) [r(s,a) + \gamma Q^{\pi}(s',a')]
= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
$$



2/ We have: 
$$
\begin{equation} 
Q^*(s,a) = \max_{\pi} Q^{\pi}(s, a) 
\end{equation}
$$
so with

$$\begin{align*} 
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \end{align*}$$

we get
$$\begin{align*} Q^*(s,a) & = E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{\pi} Q^{\pi}(s',a')] \\ & = E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^*(s',a')] \end{align*}$$

and by defination of policy we have $s' \sim \pi^*(.|s,a)$ and so

$$\begin{align*} 
Q^*(s,a) = \max_{\pi} Q^{\pi}(s,a) = E_{s' \sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')] 
\end{align*}$$

3/If we want to approximate $Q^*(s,a)$ we should define the loss as :

$$\begin{align*} 
L_i(\theta_i) = E_{s' \sim \pi^*(.|s,a)}[\Vert Q^*(s,a)- Q(s, a; \theta_i)\Vert^2] 
\end{align*}$$

and thanks to last answer, we have : 

$$\begin{align*} 
Q^*(s,a) = E_{s' \sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')] 
\end{align*}$$

Finally we have: 

\begin{align} \mathcal{L}(\theta)=E_{s' \sim \pi^(.|s,a)}[\Vert( r(s,a)+\gamma\max_{a'}Q(s',a',\theta{i-1})-Q(s,a,\theta_i))\Vert^2] \end{align}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
            self.memory.append(m)
            
    def random_access(self):
        tirage = np.random.randint(0, len(self.memory))
        return self.memory[tirage]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
          s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
          target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
          input_states[i] = s_
          
          if game_over_:
            target_q[i,a_] = r_
          else:
            q_est = self.model.predict(np.expand_dims(n_s_, 0))
            target_q[i, a_] = r_ + self.discount * np.max(q_est)
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu')) 
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.0003 | Win/lose count 1.0/4.0 (-3.0)
Epoch 001/100 | Loss 0.0026 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/100 | Loss 0.0557 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/100 | Loss 0.0184 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/100 | Loss 0.0077 | Win/lose count 1.0/2.0 (-1.0)
Epoch 005/100 | Loss 0.0102 | Win/lose count 3.0/4.0 (-1.0)
Epoch 006/100 | Loss 0.0562 | Win/lose count 3.5/5.0 (-1.5)
Epoch 007/100 | Loss 0.0118 | Win/lose count 2.5/5.0 (-2.5)
Epoch 008/100 | Loss 0.0080 | Win/lose count 2.5/5.0 (-2.5)
Epoch 009/100 | Loss 0.0147 | Win/lose count 3.0/4.0 (-1.0)
Epoch 010/100 | Loss 0.0094 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/100 | Loss 0.0043 | Win/lose count 1.5/4.0 (-2.5)
Epoch 012/100 | Loss 0.0133 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/100 | Loss 0.0035 | Win/lose count 3.5/9.0 (-5.5)
Epoch 014/100 | Loss 0.0618 | Win/lose count 1.5/3.0 (-1.5)
Epoch 015/100 | Loss 0.0358 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/100 | Loss 0.0041 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model=Sequential()
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [20]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0061 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/100 | Loss 0.0067 | Win/lose count 4.0/2.0 (2.0)
Epoch 002/100 | Loss 0.1122 | Win/lose count 2.5/3.0 (-0.5)
Epoch 003/100 | Loss 0.0067 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/100 | Loss 0.0116 | Win/lose count 1.5/1.0 (0.5)
Epoch 005/100 | Loss 0.0010 | Win/lose count 1.5/4.0 (-2.5)
Epoch 006/100 | Loss 0.0024 | Win/lose count 4.0/1.0 (3.0)
Epoch 007/100 | Loss 0.0106 | Win/lose count 3.5/5.0 (-1.5)
Epoch 008/100 | Loss 0.0023 | Win/lose count 3.0/1.0 (2.0)
Epoch 009/100 | Loss 0.0015 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/100 | Loss 0.0109 | Win/lose count 1.5/1.0 (0.5)
Epoch 011/100 | Loss 0.0044 | Win/lose count 5.0/1.0 (4.0)
Epoch 012/100 | Loss 0.0020 | Win/lose count 8.0/2.0 (6.0)
Epoch 013/100 | Loss 0.0999 | Win/lose count 4.0/0 (4.0)
Epoch 014/100 | Loss 0.0044 | Win/lose count 5.0/3.0 (2.0)
Epoch 015/100 | Loss 0.0005 | Win/lose count 5.0/4.0 (1.0)
Epoch 016/100 | Loss 0.0030 | Win/lose count 3.5/0 (3.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [21]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 11.0/2.0. Average score (9.0)
Win/lose count 6.5/3.0. Average score (6.25)
Win/lose count 10.5/3.0. Average score (6.666666666666667)
Win/lose count 8.5/3.0. Average score (6.375)
Win/lose count 9.5/1.0. Average score (6.8)
Win/lose count 13.0/1.0. Average score (7.666666666666667)
Win/lose count 12.0/1.0. Average score (8.142857142857142)
Win/lose count 14.0/3.0. Average score (8.5)
Win/lose count 4.0/2.0. Average score (7.777777777777778)
Win/lose count 12.5/3.0. Average score (7.95)
Win/lose count 5.5/0. Average score (7.7272727272727275)
Win/lose count 6.0/0. Average score (7.583333333333333)
Win/lose count 6.5/4.0. Average score (7.1923076923076925)
Win/lose count 4.5/2.0. Average score (6.857142857142857)
Win/lose count 14.0/0. Average score (7.333333333333333)
Win/lose count 14.5/3.0. Average score (7.59375)
Win/lose count 17.5/0. Average score (8.176470588235293)
Win/lose count 15.0/2.0. Average score (8.444444444444445)
Win/lose count 13.5/3.0. A

In [22]:
HTML(display_videos('cnn_test10.mp4'))

In [23]:
HTML(display_videos('fc_test10.mp4'))

The temperature increase the density of reward pixel. So it increase the results.

We observe that the rat do not explore the game and tends to be stuck and oscillate between 2 positions.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            agent.epsilon =agent.epsilon*decay
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #NEW
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        #new
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]   
        
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time 
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 
        
        self.board = bonus + malus 
        
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error, 2 times same line
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [26]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0657 | Win/lose count 11.5/48.70000000000006 (-37.20000000000006)
Epoch 001/100 | Loss 0.0108 | Win/lose count 18.0/28.300000000000015 (-10.300000000000015)
Epoch 002/100 | Loss 0.0067 | Win/lose count 23.0/13.299999999999983 (9.700000000000017)
Epoch 003/100 | Loss 0.0653 | Win/lose count 17.5/34.90000000000001 (-17.400000000000013)
Epoch 004/100 | Loss 0.0408 | Win/lose count 16.5/18.800000000000008 (-2.300000000000008)
Epoch 005/100 | Loss 0.0338 | Win/lose count 19.5/17.09999999999999 (2.4000000000000092)
Epoch 006/100 | Loss 0.0127 | Win/lose count 20.0/19.19999999999999 (0.8000000000000114)
Epoch 007/100 | Loss 0.0398 | Win/lose count 22.5/13.999999999999973 (8.500000000000027)
Epoch 008/100 | Loss 0.0225 | Win/lose count 21.0/21.699999999999996 (-0.6999999999999957)
Epoch 009/100 | Loss 0.0096 | Win/lose count 25.0/13.399999999999977 (11.600000000000023)
Epoch 010/100 | Loss 0.0121 | Win/lose count 24.0/17.09999999999998 (6.90000000000002)
Epoch 011/100 | 

In [27]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 28.5/2.0. Average score (26.5)
Win/lose count 19.0/5.0. Average score (20.25)
Win/lose count 22.5/2.0. Average score (20.333333333333332)
Win/lose count 20.5/2.0. Average score (19.875)
Win/lose count 26.0/2.0. Average score (20.7)
Win/lose count 22.5/1.0. Average score (20.833333333333332)
Win/lose count 22.0/1.0. Average score (20.857142857142858)
Win/lose count 27.5/1.0. Average score (21.5625)
Win/lose count 25.5/3.0. Average score (21.666666666666668)
Win/lose count 26.0/0. Average score (22.1)
Win/lose count 27.0/0. Average score (22.545454545454547)
Win/lose count 27.5/0. Average score (22.958333333333332)
Win/lose count 24.5/2.0. Average score (22.923076923076923)
Win/lose count 27.0/3.0. Average score (23.0)
Win/lose count 20.0/2.0. Average score (22.666666666666668)
Win/lose count 23.5/2.0. Average score (22.59375)
Win/lose count 23.0/2.0. Average score (22.5)
Win/lose count 26.0/1.0. Average score (22.63888888888889)
Win/lose count 22.5/1.0. Average score (22.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
class Mimicks(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimicks, self).__init__(epsilon = epsilon)
        self.discount = 0.99
        self.grid_size = grid_size
        self.n_state = n_state
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_[:,:,1:], a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            s_, a_  = self.memory.random_access()
            
            target_q[i, a_] = 5
            input_states[i] = s_
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
        

        
class CNN_Mimicks(Mimicks):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(CNN_Mimicks, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

def train_mimicks(ref, agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = ref.act(state, train=False)

            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
     

In [30]:
epochs_train = 20
expert = agent
agent_n = CNN_Mimicks(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimicks(expert,agent_n, env, epochs_train, prefix='cnn_train_mimicks')

Epoch 000/020 | Loss 2.7271 | Win/lose count 25.0/10.099999999999985 (14.900000000000015)
Epoch 001/020 | Loss 2.1295 | Win/lose count 27.5/20.200000000000003 (7.299999999999997)
Epoch 002/020 | Loss 2.5208 | Win/lose count 23.5/19.9 (3.6000000000000014)
Epoch 003/020 | Loss 2.4456 | Win/lose count 20.5/25.69999999999998 (-5.1999999999999815)
Epoch 004/020 | Loss 2.7402 | Win/lose count 20.5/20.599999999999987 (-0.09999999999998721)
Epoch 005/020 | Loss 2.5790 | Win/lose count 29.5/15.799999999999994 (13.700000000000006)
Epoch 006/020 | Loss 2.2746 | Win/lose count 22.0/16.999999999999993 (5.000000000000007)
Epoch 007/020 | Loss 2.8871 | Win/lose count 25.0/35.10000000000003 (-10.10000000000003)
Epoch 008/020 | Loss 2.7178 | Win/lose count 18.0/16.099999999999987 (1.9000000000000128)
Epoch 009/020 | Loss 2.8593 | Win/lose count 24.5/14.79999999999999 (9.70000000000001)
Epoch 010/020 | Loss 2.9396 | Win/lose count 17.0/18.19999999999998 (-1.1999999999999815)
Epoch 011/020 | Loss 2.2820 

In [31]:
# Evaluation
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimicks')

Win/lose count 13.5/3.0. Average score (10.5)
Win/lose count 14.5/2.0. Average score (11.5)
Win/lose count 15.0/6.0. Average score (10.666666666666666)
Win/lose count 15.5/4.0. Average score (10.875)
Win/lose count 17.0/5.0. Average score (11.1)
Win/lose count 18.0/4.0. Average score (11.583333333333334)
Win/lose count 21.0/5.0. Average score (12.214285714285714)
Win/lose count 19.5/11.0. Average score (11.75)
Win/lose count 14.5/2.0. Average score (11.833333333333334)
Win/lose count 14.0/10.0. Average score (11.05)
Win/lose count 19.5/13.0. Average score (10.636363636363637)
Win/lose count 21.0/3.0. Average score (11.25)
Win/lose count 11.5/2.0. Average score (11.115384615384615)
Win/lose count 18.5/2.0. Average score (11.5)
Win/lose count 25.0/3.0. Average score (12.2)
Win/lose count 11.0/6.0. Average score (11.75)
Win/lose count 17.0/5.0. Average score (11.764705882352942)
Win/lose count 15.0/4.0. Average score (11.722222222222221)
Win/lose count 16.0/3.0. Average score (11.78947368

This model learns faster but is not as performant as the last model.